<h2><b>Backtest</b></h2>
<h3>Teste de hipóteses para Backtest v - 0.11</h3>
<h5>v 0.11 - 2022-10-16 - Criado a coluna Rank no "Best Results"</h5>
<h4>Autor: Felipe Muller - cmtemuller@gmail.com</h4>

In [ ]:
from Backtest import Backtest_getdata, Backtest
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
import datetime
from fpdf import FPDF

### 1. Consulta Individual

#### 1.1 Get_data

In [ ]:
Ticker="PETR4.SA"            #Formato Yahoo para ações brasileiras com o ".SA" no final
Timeframe="Weekly"            #[Daily,Weekly]
DataInicial="2020-01-01"     #Formato (YYYY-mm-dd)
DataFinal="2022-12-31"       #Formato (YYYY-mm-dd)
                       #Table,Row,Summary,Graph

df = Backtest_getdata(ticker=Ticker
                      , startdate=DataInicial
                      , enddate=DataFinal
                      , timeframe=Timeframe
                      , source="yahoo")


In [ ]:
df.head(50)

#### 1.2 Consulta

In [ ]:
CaixaInicial=20000           #Em Moeda
Lote=100                     #Float
CustoCorretora=4.99          #Em Moeda
CustoB3=0.0306               #Em porcentagem
Impostos=6.5                 #Em Porcentagem
    
TradeSystem = "LegadoFinanceiro"              #"Buy", "Sell", "LegadoFinanceiro"
Setup=1

EntradaPriceType="Lowest"      # "Highest" , "Lowest", "SMA", "Close"           
EntradaPriceValue="Low"         # "Open", "High", "low", "Close"    
EntradaPriceCandles=1           # "=0" => Entrada no Fechamento, ">0" => Entrada no candle seguinte 
EntradaPriceDiffType="f"        #"P"=Percentage / "F"=Fixed
EntradaPriceDiffValue=0     #Se for porcentagem colocar 1.5 = 1.5%
EntradaCandlesStandby=0         # NOT WORKING

Filtro0=""          #"IFR;14;CTF;ABV;0,40"     
Filtro1=""       
Filtro2=""                          
Filtro3=""                          
Filtro4=""                          

TakeProfitType="Highest"               # "Conjunto 123", "Candle2","RiscoRetorno","Highest","SMA","PercentualEntrada"
TakeProfitValueCandle="High"
TakeProfitValue=4
TakeProfitPriceDiffType="F"        #"P"=Percentage / "F"=Fixed
TakeProfitPriceDiffValue=0    #Se for porcentagem colocar 1.5 = 1.5%

StopLossType="Lowest"              # "Highest" , "Lowest", "SMA"                
StopLossValueCandle="Low"
StopLossValue=10
StopLossFixed = False             #True or False
StopLossPriceDiffType="F"        #"P"=Percentage / "F"=Fixed
StopLossPriceDiffValue=+0.00     #Se for porcentagem colocar 1.5 = 1.5%

Tempo=0                          #Integer, se 0=Sem Stop no Tempo

bt = Backtest(df
              , Timeframe
              , DataInicial
              , DataFinal
              , CaixaInicial
              , Lote
              , CustoCorretora
              , CustoB3
              , Impostos
              , Filtro0
              , Filtro1
              , Filtro2
              , Filtro3
              , Filtro4
              , TradeSystem
              , Setup
              , EntradaPriceType
              , EntradaPriceValue
              , EntradaPriceCandles
              , EntradaPriceDiffType
              , EntradaPriceDiffValue
              , EntradaCandlesStandby
              , TakeProfitType
              , TakeProfitValueCandle
              , TakeProfitValue
              , TakeProfitPriceDiffType
              , TakeProfitPriceDiffValue
              , StopLossType
              , StopLossValueCandle
              , StopLossValue
              , StopLossFixed
              , StopLossPriceDiffType
              , StopLossPriceDiffValue
              , Tempo
              , Retorno='graph_save')
bt

In [ ]:
#bt.to_excel(r'C:\Users\cmtem\OneDrive\Área de Trabalho\backtest_sorted_'+Ticker+'_'+Timeframe+'.xls', index = True)

### 2 Teste de Hipóteses

#### 2.0 Dados

In [ ]:
Ticker="PETR4.SA"     
source = "yahoo"

CaixaInicial=20000           #Em Moeda
Lote=100                     #Float
CustoCorretora=4.99          #Em Moeda
CustoB3=0.0306               #Em porcentagem
Impostos=6.5   

DataInicial=["2012-01-01","2017-01-01","2020-01-01"]
DataFinal="2022-12-31"

Timeframe='Weekly' 

TradeSystem = "LegadoFinanceiro"
Setup = ""


#### 2.1 Hipótese 1: 
Melhor Backtest do 3 Bar High Low, modificando Filtro(periodo), 

In [ ]:
EntradaPriceType=['Highest','Lowest','SMA']                      #["Highest","Lowest"]           
EntradaPriceValue=['Low']
EntradaPriceCandles=[1,2,3,4,5]                           #[1,2,3,4,5]
EntradaPriceDiffType='F'
EntradaPriceDiffValue=[0]
EntradaCandlesStandby=''

Filtro0Value=[1,2,3,4,9]                                         #[2,3,4,5,8,10,15,20]  

TakeProfitType=['Highest','Lowest','SMA']                          #["Highest","SMA"]
TakeProfitValueCandle=['High']
TakeProfitValue=[1,2,3,4,5]                                    #[1,2,3,4,5]
TakeProfitPriceDiffType='F'
TakeProfitPriceDiffValue=[0]

StopLossType="None"
StopLossValueCandle="Low"
StopLossValue=10
StopLossFixed = True                                  #True or False
StopLossPriceDiffType="F"
StopLossPriceDiffValue=+0.00 

Tempo=[1,2,3,4,5,6]                                                #[0,1,2,3,4,5]

#----------------------------------------------------------------------------------------#
bt_all=[]
i=0
StartTime = datetime.datetime.now()     
TotalInteractions = len(DataInicial)*len(EntradaPriceType)*len(EntradaPriceValue)*len(EntradaPriceCandles)*len(EntradaPriceDiffValue)*len(Filtro0Value)*len(TakeProfitType)*len(TakeProfitValueCandle)*len(TakeProfitValue)*len(TakeProfitPriceDiffValue)*len(Tempo)
print("Interações Totais: "+str(TotalInteractions))
print("-----")
for DI in DataInicial:
    df = Backtest_getdata(ticker=Ticker
                      , startdate=DI
                      , enddate=DataFinal
                      , timeframe=Timeframe
                      , source="yahoo")
    for EPT in EntradaPriceType:
        for EPV in EntradaPriceValue:
            for EPC in EntradaPriceCandles: 
                for EPDV in EntradaPriceDiffValue:
                    for F in Filtro0Value:
                        for TPT in TakeProfitType:
                            for TPVC in TakeProfitValueCandle:
                                for TPV in TakeProfitValue: 
                                    for TPPDV in TakeProfitPriceDiffValue:
                                        for T in Tempo:
                                            print(str(i)+": DataInicial: "+DI+" ")
                                            print ("  Entrada: "+EPT+";"+EPV+";"+str(EPC)+";"+EntradaPriceDiffType+";"+str(EPDV))
                                            print ("  Filtro0: SMA;"+str(F)+";CTF;ABV&ASC")
                                            print ("  TakeProfit: "+TPT+";"+TPVC+";"+str(TPV)+";"+TakeProfitPriceDiffType+";"+str(TPPDV))
                                            print ("  Tempo:"+str(T))
                                            bt = Backtest(df
                                                  , TimeFrame = Timeframe
                                                  , CaixaInicial = CaixaInicial
                                                  , DataInicial = DI
                                                  , DataFinal = DataFinal
                                                  , Lote = Lote
                                                  , CustoCorretora = CustoCorretora
                                                  , CustoB3 = CustoB3
                                                  , Impostos = Impostos
                                                  , Filtro0 = "SMA;"+str(F)+";CTF;ABV"
                                                  , Filtro1 = ""
                                                  , Filtro2 = ""
                                                  , Filtro3 = ""
                                                  , Filtro4 = ""
                                                  , TradeSystem = TradeSystem
                                                  , Setup = Setup
                                                  , EntradaPriceType = EPT
                                                  , EntradaPriceValue = EPV
                                                  , EntradaPriceCandles = EPC
                                                  , EntradaPriceDiffType = EntradaPriceDiffType
                                                  , EntradaPriceDiffValue = EPDV 
                                                  , EntradaCandlesStandby = 0
                                                  , TakeProfitType = TPT
                                                  , TakeProfitValueCandle = TPVC
                                                  , TakeProfitValue = TPV
                                                  , TakeProfitPriceDiffType = TakeProfitPriceDiffType
                                                  , TakeProfitPriceDiffValue = TPPDV 
                                                  , StopLossType = StopLossType
                                                  , StopLossValueCandle = StopLossValueCandle
                                                  , StopLossValue = StopLossValue
                                                  , StopLossFixed = StopLossFixed        
                                                  , StopLossPriceDiffType = StopLossPriceDiffType
                                                  , StopLossPriceDiffValue = StopLossPriceDiffValue
                                                  , Tempo = T
                                                  , Retorno='row')
                                            i=i+1
                                            ActualTime = datetime.datetime.now()
                                            delta = (ActualTime - StartTime)/i
                                            # time difference in seconds
                                            print(f"Time to calculate(mean)): {delta.total_seconds()} seconds")
                                            print ("Remaining Time to End: "+str(round((delta.total_seconds()*(TotalInteractions-i))/60,1))+" minutes")
                                            print("---")
                                            OldTime = datetime.datetime.now()
                                            try:
                                                bt_all = pd.concat([bt_all, bt], ignore_index=True)
                                            except: 
                                                bt_all = bt.copy()

deltaTotal = (ActualTime - StartTime)
# time difference in seconds
print(f"Total Time: {round(deltaTotal.total_seconds()/60,1)} minutes")
                                            

In [ ]:
bt_sorted = bt_all.copy()
bt_sorted['Rank']=0
Datas = bt_sorted.groupby(by=["DataInicial"]).sum().index
i=0

for i in range(len(Datas)-1):
    bt_rank = bt_sorted[bt_sorted['DataInicial']==Datas[i].strftime("%Y-%m-%d")]
    bt_rank = bt_rank.sort_values(['LucroAcoes'], axis=0, ascending=False, inplace=False, ignore_index=False)
    bt_rank.reset_index(inplace=True, drop=False)
    bt_rank.index = bt_rank.index+1
    j = 0
    for j in range(bt_rank.shape[0]):
        bt_sorted['Rank'][bt_rank['index']]=bt_rank.index
        j=j+1

bt_sorted = bt_sorted.groupby(['Entrada','Filtro 0','TakeProfit','StopTempo']).agg(
    {'LucroAcoes': 'sum',
    'ResulLiquidoPerc':'sum',
    'TotalTrades':'sum',
    'WinRate':'mean',
    'MediaPercCandle':'mean', 
    'Rank':'sum'})
bt_sorted.reset_index(inplace=True)
bt_sorted = bt_sorted.sort_values(['LucroAcoes'], axis=0, ascending=False, inplace=False, ignore_index=False)
bt_sorted.reset_index(inplace=True, drop=True)
bt_sorted['ResulLiquidoPerc']=round(bt_sorted['ResulLiquidoPerc']*100,2)
bt_sorted['WinRate']=round(bt_sorted['WinRate']*100,2)
bt_sorted['MediaPercCandle']=round(bt_sorted['MediaPercCandle']*100,2)

bt_sorted.head(50)

### CSV

In [ ]:
bt_all.to_csv(r'C:\Users\cmtem\OneDrive\Área de Trabalho\backtest_all_'+Ticker+'_'+Timeframe+'.csv', index = False)
bt_sorted.to_csv(r'C:\Users\cmtem\OneDrive\Área de Trabalho\backtest_sorted_'+Ticker+'_'+Timeframe+'.csv', index = True)

### Best Results

In [ ]:
indice = 0      

df_br = Backtest_getdata(ticker=Ticker
                      , startdate=DataInicial[0]
                      , enddate=DataFinal
                      , timeframe=Timeframe
                      , source="yahoo")
bt_br_summary = Backtest(df_br
              , TimeFrame = Timeframe
              , DataInicial = DataInicial[0] 
              , DataFinal = DataFinal
              , CaixaInicial = CaixaInicial
              , Lote = Lote
              , CustoCorretora = CustoCorretora
              , CustoB3 = CustoB3
              , Impostos = Impostos
              , Filtro0 = bt_sorted['Filtro 0'][indice]
              , Filtro1 = ""
              , Filtro2 = ""
              , Filtro3 = ""
              , Filtro4 = ""
              , TradeSystem = TradeSystem
              , Setup = Setup
              , EntradaPriceType = bt_sorted['Entrada'].str.split(';').str[0][indice]
              , EntradaPriceValue = bt_sorted['Entrada'].str.split(';').str[1][indice]
              , EntradaPriceCandles = int(bt_sorted['Entrada'].str.split(';').str[2][indice])
              , EntradaPriceDiffType = bt_sorted['Entrada'].str.split(';').str[3][indice]
              , EntradaPriceDiffValue = int(bt_sorted['Entrada'].str.split(';').str[4][indice])
              , EntradaCandlesStandby = 0
              , TakeProfitType = bt_sorted['TakeProfit'].str.split(';').str[0][indice]
              , TakeProfitValueCandle = bt_sorted['TakeProfit'].str.split(';').str[1][indice]
              , TakeProfitValue = int(bt_sorted['TakeProfit'].str.split(';').str[2][indice])
              , TakeProfitPriceDiffType = bt_sorted['TakeProfit'].str.split(';').str[3][indice]
              , TakeProfitPriceDiffValue = int(bt_sorted['TakeProfit'].str.split(';').str[4][indice])
              , StopLossType = ""
              , StopLossValueCandle = ""
              , StopLossValue = ""
              , StopLossPriceDiffType = ""
              , StopLossPriceDiffValue = ""
              , Tempo = int(bt_sorted['StopTempo'][indice])
              , Retorno='summary')

bt_br_summary

In [ ]:
bt_br_graph = Backtest(df_br
              , TimeFrame = Timeframe
              , DataInicial = DataInicial[0] 
              , DataFinal = DataFinal
              , CaixaInicial = CaixaInicial
              , Lote = Lote
              , CustoCorretora = CustoCorretora
              , CustoB3 = CustoB3
              , Impostos = Impostos
              , Filtro0 = bt_sorted['Filtro 0'][indice]
              , Filtro1 = ""
              , Filtro2 = ""
              , Filtro3 = ""
              , Filtro4 = ""
              , TradeSystem = TradeSystem
              , Setup = Setup
              , EntradaPriceType = bt_sorted['Entrada'].str.split(';').str[0][indice]
              , EntradaPriceValue = bt_sorted['Entrada'].str.split(';').str[1][indice]
              , EntradaPriceCandles = int(bt_sorted['Entrada'].str.split(';').str[2][indice])
              , EntradaPriceDiffType = bt_sorted['Entrada'].str.split(';').str[3][indice]
              , EntradaPriceDiffValue = int(bt_sorted['Entrada'].str.split(';').str[4][indice])
              , EntradaCandlesStandby = 0
              , TakeProfitType = bt_sorted['TakeProfit'].str.split(';').str[0][indice]
              , TakeProfitValueCandle = bt_sorted['TakeProfit'].str.split(';').str[1][indice]
              , TakeProfitValue = int(bt_sorted['TakeProfit'].str.split(';').str[2][indice])
              , TakeProfitPriceDiffType = bt_sorted['TakeProfit'].str.split(';').str[3][indice]
              , TakeProfitPriceDiffValue = int(bt_sorted['TakeProfit'].str.split(';').str[4][indice])
              , StopLossType = ""
              , StopLossValueCandle = ""
              , StopLossValue = ""
              , StopLossPriceDiffType = ""
              , StopLossPriceDiffValue = ""
              , Tempo = int(bt_sorted['StopTempo'][indice])
              , Retorno='graph')
bt_br_graph

In [ ]:
bt_br_graph_save = Backtest(df_br
              , TimeFrame = Timeframe
              , DataInicial = DataInicial[0] 
              , DataFinal = DataFinal
              , CaixaInicial = CaixaInicial
              , Lote = Lote
              , CustoCorretora = CustoCorretora
              , CustoB3 = CustoB3
              , Impostos = Impostos
              , Filtro0 = bt_sorted['Filtro 0'][indice]
              , Filtro1 = ""
              , Filtro2 = ""
              , Filtro3 = ""
              , Filtro4 = ""
              , TradeSystem = TradeSystem
              , Setup = Setup
              , EntradaPriceType = bt_sorted['Entrada'].str.split(';').str[0][indice]
              , EntradaPriceValue = bt_sorted['Entrada'].str.split(';').str[1][indice]
              , EntradaPriceCandles = int(bt_sorted['Entrada'].str.split(';').str[2][indice])
              , EntradaPriceDiffType = bt_sorted['Entrada'].str.split(';').str[3][indice]
              , EntradaPriceDiffValue = int(bt_sorted['Entrada'].str.split(';').str[4][indice])
              , EntradaCandlesStandby = 0
              , TakeProfitType = bt_sorted['TakeProfit'].str.split(';').str[0][indice]
              , TakeProfitValueCandle = bt_sorted['TakeProfit'].str.split(';').str[1][indice]
              , TakeProfitValue = int(bt_sorted['TakeProfit'].str.split(';').str[2][indice])
              , TakeProfitPriceDiffType = bt_sorted['TakeProfit'].str.split(';').str[3][indice]
              , TakeProfitPriceDiffValue = int(bt_sorted['TakeProfit'].str.split(';').str[4][indice])
              , StopLossType = ""
              , StopLossValueCandle = ""
              , StopLossValue = ""
              , StopLossPriceDiffType = ""
              , StopLossPriceDiffValue = ""
              , Tempo = int(bt_sorted['StopTempo'][indice])
              , Retorno='graph_save')

In [ ]:
bt_br_trades = Backtest(df_br
              , TimeFrame = Timeframe
              , DataInicial = DataInicial[0] 
              , DataFinal = DataFinal
              , CaixaInicial = CaixaInicial
              , Lote = Lote
              , CustoCorretora = CustoCorretora
              , CustoB3 = CustoB3
              , Impostos = Impostos
              , Filtro0 = bt_sorted['Filtro 0'][indice]
              , Filtro1 = ""
              , Filtro2 = ""
              , Filtro3 = ""
              , Filtro4 = ""
              , TradeSystem = TradeSystem
              , Setup = Setup
              , EntradaPriceType = bt_sorted['Entrada'].str.split(';').str[0][indice]
              , EntradaPriceValue = bt_sorted['Entrada'].str.split(';').str[1][indice]
              , EntradaPriceCandles = int(bt_sorted['Entrada'].str.split(';').str[2][indice])
              , EntradaPriceDiffType = bt_sorted['Entrada'].str.split(';').str[3][indice]
              , EntradaPriceDiffValue = int(bt_sorted['Entrada'].str.split(';').str[4][indice])
              , EntradaCandlesStandby = 0
              , TakeProfitType = bt_sorted['TakeProfit'].str.split(';').str[0][indice]
              , TakeProfitValueCandle = bt_sorted['TakeProfit'].str.split(';').str[1][indice]
              , TakeProfitValue = int(bt_sorted['TakeProfit'].str.split(';').str[2][indice])
              , TakeProfitPriceDiffType = bt_sorted['TakeProfit'].str.split(';').str[3][indice]
              , TakeProfitPriceDiffValue = int(bt_sorted['TakeProfit'].str.split(';').str[4][indice])
              , StopLossType = ""
              , StopLossValueCandle = ""
              , StopLossValue = ""
              , StopLossPriceDiffType = ""
              , StopLossPriceDiffValue = ""
              , Tempo = int(bt_sorted['StopTempo'][indice])
              , Retorno='trades')
bt_br_trades

In [ ]:
pdf = FPDF('P','mm', 'A4')
pdf.add_page()
pdf.set_font("Arial", size=14)

pdf.cell(200, 30, "Relatório Backtest: "+str(Ticker), border=0)
pdf.ln(20)        
#----Página 1-----
bt_sorted_pdf = bt_sorted.head(50)
w = [8,20,20,20,15,15,20,15,15,19,10]
h = 5
i = 0
for index in bt_sorted_pdf.index:
    if i == 0:
        c=1
        pdf.set_font("Arial", "B", size=6)
        pdf.cell(w[0], h, "", border=1, align = 'C')
        for columns in bt_sorted_pdf.columns:
            pdf.cell(w[c], h, columns, border=1, align = 'C')
            c=c+1
        i = i+1
    else:
        c=1
        pdf.set_font("Arial", "B", size=6)
        pdf.cell(w[0], h, str(index), border=1, align = 'C')
        for columns in bt_sorted_pdf.columns:
            pdf.set_font("Arial", "", size=6)
            pdf.cell(w[c], h, str(bt_sorted_pdf[columns][index]), border=1, align = 'C')
            c=c+1
    pdf.ln(h)

#----Página 2-----
pdf.add_page()
pdf.set_font("Arial", size=14)
pdf.cell(200, 30, "Melhor Resultado: ", border=0)
pdf.ln(20) 
pdf.set_font("Arial", size=7)
w = [45,30,25,60]
h = 5
i = 0

for index in bt_br_summary.index:
    if i == 0:
        c = 1
        pdf.set_font("Arial", 'B', 7)
        pdf.cell(w[0], h, "", border=1, align = 'C')
        for columns in bt_br_summary.columns:
            pdf.cell(w[c], h, columns, border=1, align = 'C')
            c = c+1
        i = i+1
    else:
        c = 1
        pdf.set_font("Arial", 'B', 7)
        pdf.cell(w[0], h, index, border=1, align = 'C')
        for columns in bt_br_summary.columns:
            pdf.set_font("Arial", '', 7)
            pdf.cell(w[c], h, str(bt_br_summary[columns][index]), border=1, align = 'C')
            c = c+1
            
    pdf.ln(h)

pdf.image(r'C:\Users\cmtem\OneDrive\Área de Trabalho\fig1.png', w = 200, h= 240, type = 'png' )

#----Página 4-----
pdf.add_page('P')
bt_br_trades_pdf=bt_br_trades[["Open","High","Low","Close",'Entrada','TakeProfit','StopLoss','Trade','Perc']].copy()
bt_br_trades_pdf['Open']=round(bt_br_trades['Open'],2)
bt_br_trades_pdf['High']=round(bt_br_trades['High'],2)
bt_br_trades_pdf['Low']=round(bt_br_trades['Low'],2)
bt_br_trades_pdf['Close']=round(bt_br_trades['Close'],2)
bt_br_trades_pdf['Entrada']=round(bt_br_trades['Entrada'],2)
bt_br_trades_pdf['TakeProfit']=round(bt_br_trades['TakeProfit'],2)
bt_br_trades_pdf['StopLoss']=round(bt_br_trades['StopLoss'],2)
bt_br_trades_pdf['Trade']=bt_br_trades['Trade']
bt_br_trades_pdf['Perc']=round(bt_br_trades['Perc']*100,2)
w = [25,15,15,15,15,15,15,15,15,15]
h = 5
i = 0
for index in bt_br_trades_pdf.index:
    if i == 0:
        c = 1
        pdf.set_font("Arial", 'B', 7)
        pdf.cell(w[0], h, "", border=1, align='C')
        for columns in bt_br_trades_pdf.columns:
            pdf.cell(w[c], h, columns, border=1, align = 'C')
            c = c+1
        i = i+1
    else:
        c = 1
        pdf.set_font("Arial", 'B', 7)
        pdf.cell(w[0], h, str(index), border=1, align='c')
        for columns in bt_br_trades_pdf.columns:
            pdf.set_font("Arial", '', 7)
            pdf.cell(w[c], h, str(bt_br_trades_pdf[columns][index]), border=1, align = 'C')
            c = c+1
            
    pdf.ln(h)
    

pdf.output(r'C:\Users\cmtem\OneDrive\Área de Trabalho\relatorio'+Ticker+'-'+Timeframe+'.pdf', 'F')